In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from pymongo import MongoClient
from dotenv import load_dotenv

In [2]:
# Nama database dan collection
db_name = 'youtube_rewind_indonesia'
collection_label = 'labeling_2015'
collection_name = 'full_labeling_2015'

# Memuat value dari file .env
load_dotenv()

mongodb_url = os.getenv('URL_SANDY')
local_url = os.getenv('URL_LOCAL')

#### Mengambil data dari MongoDB ke dalam dataframe

In [3]:
# Membuat koneksi ke MongoDB
client = MongoClient(local_url)
db = client[db_name]  # Ganti dengan nama database Anda
labeling_2015_collection = db[collection_label]
full_labeling_2015_collection = db[collection_name]

# Mengambil data dari koleksi MongoDB
labeled_data = list(labeling_2015_collection.find())
unlabeled_data = list(full_labeling_2015_collection.find())

# Konversi data menjadi dataframe
df_labeled = pd.DataFrame(labeled_data)
df_unlabeled = pd.DataFrame(unlabeled_data)

# Memisahkan fitur dan label dari data yang berlabel
X_labeled = df_labeled['textOriginal']
y_labeled = df_labeled['label']

# Menggunakan TfidfVectorizer untuk mengubah teks menjadi fitur
vectorizer = TfidfVectorizer()
X_labeled_tfidf = vectorizer.fit_transform(X_labeled)

# Melatih model menggunakan Naive Bayes
model = MultinomialNB()
model.fit(X_labeled_tfidf, y_labeled)

# Memproses data yang tidak berlabel
X_unlabeled = df_unlabeled['textOriginal']
X_unlabeled_tfidf = vectorizer.transform(X_unlabeled)

# Memprediksi label untuk data yang tidak berlabel
y_unlabeled_pred = model.predict(X_unlabeled_tfidf)

# Menambahkan prediksi ke dalam dataframe data tidak berlabel
df_unlabeled['predicted_label'] = y_unlabeled_pred

# Mengupdate koleksi MongoDB dengan prediksi label
for index, row in df_unlabeled.iterrows():
    full_labeling_2015_collection.update_one(
        {"_id": row["_id"]},
        {"$set": {"predicted_label": row["predicted_label"]}}
    )

print("Prediksi label untuk data tidak berlabel telah selesai dan diupdate dalam koleksi MongoDB.")

Prediksi label untuk data tidak berlabel telah selesai dan diupdate dalam koleksi MongoDB.


#### Accuracy Test (soon)

In [4]:
# Jika data tidak berlabel memiliki label asli untuk keperluan evaluasi
if 'label' in df_unlabeled.columns:
    print("\nEvaluasi kinerja model:")
    print("Accuracy Score:", accuracy_score(df_unlabeled['label'], df_unlabeled['predicted_label']))
    print("Classification Report:\n", classification_report(df_unlabeled['label'], df_unlabeled['predicted_label']))
else:
    print("Data tidak berlabel tidak memiliki label asli untuk evaluasi.")

Data tidak berlabel tidak memiliki label asli untuk evaluasi.
